# Title: Severity of road traffic accidents

In [ ]:
%load_ext watermark
%watermark -a "Yifan Wu" -u -d -t -v -p numpy,pandas,matplotlib,scikit-learn

## Preparation

- [Github link](https://github.com/Van-Wu1/0006.git)

- Number of words: ***

- Runtime: *** hours (*Memory 32 GB, CPU AMD Ryzen 7 5800H with Radeon Graphics CPU @3.20GHz*)

- Coding environment: Coding environment: VS Code with Jupyter plugin (local), not SDS Docker

- License: this notebook is made available under the [Creative Commons Attribution license](https://creativecommons.org/licenses/by/4.0/).

- Additional library *[libraries not included in SDS Docker or not used in this module]*:
    - **watermark**: A Jupyter Notebook extension for printing timestamps, version numbers, and hardware information.(used to print Python and package versions for reproducibility.)
    - **osmnx**: For downloading and analyzing OpenStreetMap road network data.
    - **networkx**: For calculating road network metrics such as betweenness and degree centrality.
    - **geopandas**: For spatial data handling, including reading GeoJSON borough boundaries.
    - **shap**: For model interpretability using SHAP value analysis.
    - **xgboost**: For gradient boosting machine learning classification.
    - **tqdm**: For displaying progress bars during borough-level computations.


## Table of contents

1. [Introduction](#Introduction)

1. [Research questions](#Research-questions)

1. [Data](#Data)

1. [Methodology](#Methodology)

1. [Results and discussion](#Results-and-discussion)

1. [Conclusion](#Conclusion)

1. [References](#References)

## Introduction

[[ go back to the top ]](#Table-of-contents)

Road traffic accidents (RTAs) represent a significant public health and urban governance issue globally. In the UK, despite advancements in vehicle technology and traffic regulation, thousands of individuals are injured or killed on the roads annually. Predicting the severity of these accidents is crucial for targeted policy interventions and infrastructure planning. Accident severity is influenced by a range of contextual factors including weather, road geometry, traffic volume, time of day, and infrastructure design (Abdel-Aty & Haleem, 2011). As cities move towards data-driven governance, the application of machine learning methods to road safety research has become increasingly common, offering more accurate and interpretable approaches to understanding complex risk patterns (Ahmed et al., 2023).

Recent literature has demonstrated the effectiveness of supervised learning algorithms such as logistic regression, random forests, and XGBoost in predicting accident severity using structured datasets (Ahmed et al., 2023). These models are particularly suitable for capturing non-linear interactions and heterogeneous effects among multiple explanatory variables. Moreover, explainable AI techniques such as SHAP (SHapley Additive exPlanations) have been widely adopted to interpret complex models and understand feature importance, which aids in translating statistical findings into actionable insights for policymakers.

This study leverages the UK Department for Transport’s Road Safety Data (2015–2019), which documents detailed information on individual accident cases, including temporal, spatial, environmental, and infrastructural attributes. By integrating network-based features such as road betweenness centrality extracted via OpenStreetMap, this project attempts to bridge the gap between spatial network analysis and predictive modelling of accident severity. The objective is twofold: to evaluate the predictive performance of commonly used machine learning models on accident severity classification, and to examine the relative contribution of different spatial and contextual factors to the outcome.

The period from 2015 to 2019 was deliberately chosen to ensure data stability and validity. This timeframe avoids the confounding effects of the COVID-19 pandemic (2020–2021), which significantly disrupted travel behaviour, enforcement levels, and urban mobility patterns across the UK (DfT, 2021). It also precedes Phase 2 of London's major road transformation programme, including the expansion of Low Traffic Neighbourhoods (LTNs) and segregated cycling infrastructure, which introduced substantial structural changes to the transport system from 2020 onwards (TfL, 2024). In contrast, the preceding years (2010–2014) marked a foundational policy phase, characterised by the implementation of new traffic enforcement measures such as fixed penalty notices for careless driving and increased fines for common violations (DfT, 2013). By focusing on the relatively stable and mature period between 2015 and 2019, this study ensures greater internal consistency and enables clearer interpretation of accident severity patterns, isolated from exogenous policy or behavioural shocks.

In contrast to most existing studies that primarily rely on tabular attributes such as time, weather, and road conditions, this project introduces spatial network metrics—specifically, betweenness and degree centrality—extracted from OpenStreetMap. This integration of spatial topology enhances the model's capacity to capture urban structural influences on accident severity, offering a novel bridge between road network analysis and predictive modeling.

## Research questions

[[ go back to the top ]](#Table-of-contents)

Can supervised machine learning models accurately predict the severity of road traffic accidents in London using spatial, temporal, and environmental features?

This project explores whether supervised machine learning models can accurately predict the severity of road traffic accidents in London based on spatial, temporal, and environmental features. Specifically, it evaluates the contribution of variables such as time of day, weather conditions, and road network centrality. The study compares the performance of Logistic Regression, Random Forest, and XGBoost classifiers, and employs SHAP (SHapley Additive exPlanations) to interpret model outputs and quantify feature importance across different severity levels (fatal, serious, slight).

## Data

[[ go back to the top ]](#Table-of-contents)

### Data Description

| Variable                                 | Type         | Description                                                                                  | Notes                                 |
|------------------------------------------|--------------|----------------------------------------------------------------------------------------------|----------------------------------------|
| accident_severity                        | Categorical  | Severity level of the accident (1 = Fatal, 2 = Serious, 3 = Slight)                          | Target variable                        |
| speed_limit                              | Numeric      | Speed limit of the road segment (in mph)                                                     | -                                      |
| accident_year                            | Numeric      | Year of the accident (2015–2019)                                                             | Used for train-test split              |
| mean_betweenness                         | Numeric      | Mean betweenness centrality of nearby road segments                                          | Spatial network feature                |
| max_betweenness                          | Numeric      | Maximum betweenness centrality of nearby road segments                                       | Key spatial variable                   |
| mean_degree                              | Numeric      | Mean degree centrality of road network                                                       | -                                      |
| max_degree                               | Numeric      | Maximum degree centrality                                                                    | -                                      |
| edge_count                               | Numeric      | Number of road segments (edges) in the local road network                                    | Spatial indicator of network density   |
| day_of_week_*                            | Categorical  | One-hot encoded day of week (Monday–Saturday, Sunday as baseline)                           | One-hot encoded                        |
| road_type_*                              | Categorical  | One-hot encoded road type categories                                                         | One-hot encoded                        |
| light_conditions_*                       | Categorical  | One-hot encoded lighting conditions (e.g., daylight, darkness with/without lighting)         | One-hot encoded                        |
| weather_conditions_*                     | Categorical  | One-hot encoded weather conditions (e.g., fine, rain, fog)                                   | One-hot encoded                        |
| road_surface_conditions_*                | Categorical  | One-hot encoded surface conditions (e.g., dry, wet)                                          | One-hot encoded                        |
| junction_control_*                       | Categorical  | One-hot encoded control types at junctions                                                   | One-hot encoded                        |
| junction_detail_*                        | Categorical  | One-hot encoded structural junction types                                                    | One-hot encoded                        |
| pedestrian_crossing_human_control_*      | Categorical  | One-hot encoded presence of human-controlled crossings                                       | One-hot encoded                        |
| pedestrian_crossing_physical_facilities_*| Categorical  | One-hot encoded presence of physical pedestrian facilities                                   | One-hot encoded                        |
| special_conditions_at_site_*             | Categorical  | One-hot encoded site-specific conditions (e.g., roadworks)                                   | One-hot encoded                        |
| first_road_class_*                       | Categorical  | One-hot encoded classification of the primary road                                           | One-hot encoded                        |
| second_road_class_*                      | Categorical  | One-hot encoded classification of the secondary road                                         | One-hot encoded                        |
| trunk_road_flag_*                        | Categorical  | One-hot encoded trunk road indicator                                                         | One-hot encoded                        |
| urban_or_rural_area_*                    | Categorical  | One-hot encoded urban/rural area classification                                              | One-hot encoded                        |
| time_hour                                | Numeric      | Hour of the accident (e.g., 13:55 → 13)                                                      | Derived feature                        |
| betweenness_level_encoded                | Ordinal      | Quartile level of mean_betweenness (0 = Low, 3 = High)                                       | For logistic regression compatibility  |
| ......  | ......  | ......                                    |   |


> *Note: `*_` denotes one-hot encoded categories split into multiple columns.*


The following table provides code-level descriptions for categorical variables used in this study. Definitions are based on the official UK Department for Transport data guide: [data.gov.uk](https://www.data.gov.uk/dataset/cb7ae6f0-4be6-4935-9277-47e5ce24a11f).

| Variable Prefix                         | Code | Meaning                                   |
|----------------------------------------|------|-------------------------------------------|
| day_of_week                            | 1    | Sunday                                    |
|                                        | 2    | Monday                                    |
|                                        | 3    | Tuesday                                   |
|                                        | 4    | Wednesday                                 |
|                                        | 5    | Thursday                                  |
|                                        | 6    | Friday                                    |
|                                        | 7    | Saturday                                  |
| road_type                              | 1    | Roundabout                                |
|                                        | 2    | One way street                            |
|                                        | 3    | Dual carriageway                          |
|                                        | 6    | Single carriageway                        |
|                                        | 7    | Slip road                                  |
|                                        | 9    | Unknown                                   |
| light_conditions                       | 1    | Daylight                                   |
|                                        | 4    | Darkness - lights lit                      |
|                                        | 5    | Darkness - lights unlit                    |
|                                        | 6    | Darkness - no lighting                     |
|                                        | 7    | Darkness - lighting unknown                |
| weather_conditions                     | 1    | Fine no high winds                         |
|                                        | 2    | Raining no high winds                      |
|                                        | 3    | Snowing no high winds                      |
|                                        | 4    | Fine + high winds                          |
|                                        | 5    | Raining + high winds                       |
|                                        | 6    | Snowing + high winds                       |
|                                        | 7    | Fog or mist                                |
|                                        | 8    | Other                                      |
|                                        | 9    | Unknown                                    |
| road_surface_conditions                | 1    | Dry                                        |
|                                        | 2    | Wet or damp                                |
|                                        | 3    | Snow                                       |
|                                        | 4    | Frost or ice                               |
|                                        | 5    | Flood (surface water)                      |
|                                        | 9    | Unknown                                    |
| junction_control                       | 0    | None                                       |
|                                        | 1    | Authorised person                          |
|                                        | 2    | Auto traffic signal                        |
|                                        | 3    | Stop sign                                  |
|                                        | 4    | Give way or uncontrolled                   |
|                                        | 9    | Unknown                                    |
| pedestrian_crossing_human_control      | 0    | None                                       |
|                                        | 1    | School crossing patrol                     |
|                                        | 2    | Other human control                        |
|                                        | 9    | Unknown                                    |
| pedestrian_crossing_physical_facilities| 0    | None                                       |
|                                        | 1    | Zebra crossing                             |
|                                        | 4    | Pelican crossing                           |
|                                        | 5    | Footbridge or subway                       |
|                                        | 7    | Refuge                                     |
|                                        | 8    | Unknown                                     |
|                                        | 9    | Other                                      |
| urban_or_rural_area                    | 1    | Not used                                   |
|                                        | 2    | Urban                                      |
|                                        | 3    | Rural                                      |
| trunk_road_flag                        | 1    | Non-trunk road                             |
|                                        | 2    | Trunk road                                 |
| first_road_class / second_road_class   | 1    | Motorway                                   |
|                                        | 2    | A(M) Road                                   |
|                                        | 3    | A Road                                     |
|                                        | 4    | B Road                                     |
|                                        | 5    | C Road                                     |
|                                        | 6    | Unclassified                               |


### Data Import & Cleaning

In [ ]:
# It would import the packages that would be used first. 
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import os
import osmnx as ox
import networkx as nx
import geopandas as gpd
from tqdm import tqdm

In [ ]:
# define folder
input_folder = '../data/raw'
output_folder = '../data/clean'

In [ ]:
# Road Data
df = pd.read_csv('../data/raw/1979-latest-published-year.csv')
df = df[df['accident_year'].isin([2015, 2016, 2017, 2018, 2019])]
print(f"The data volume from 2015 to 2019 is:{len(df)} ")

# save
df.to_csv("../data/raw/2015_2019.csv", index=False)

In [ ]:
columns_to_keep = [
    'accident_severity',
    'number_of_vehicles',
    'number_of_casualties',
    'day_of_week',
    'time',
    'first_road_class',
    'second_road_class',
    'road_type',
    'speed_limit',
    'junction_detail',
    'junction_control',
    'pedestrian_crossing_human_control',
    'pedestrian_crossing_physical_facilities',
    'light_conditions',
    'weather_conditions',
    'road_surface_conditions',
    'special_conditions_at_site',
    'carriageway_hazards',
    'urban_or_rural_area',
    'did_police_officer_attend_scene_of_accident',
    'trunk_road_flag',
    'local_authority_ons_district',
    'accident_year'
]

selected_columns = [col for col in columns_to_keep if col in df.columns]
df_cleaned = df[selected_columns]

# Check and handle the missing values
missing_counts = df_cleaned.isnull().sum()
total_missing = missing_counts.sum()

if total_missing > 0:
    print(f"The number of missing values are {total_missing} :")
    print(missing_counts[missing_counts > 0])
    
    # Discard the rows containing missing values
    df_cleaned = df_cleaned.dropna()
    print(f"Missing values have been cleared, remaining {len(df_cleaned)} records.")

# Save the cleaned files
df_cleaned.to_csv('../data/clean/1519_cleaned.csv', index=False)
print(f"Saved to: {output_folder}, total: {len(df_cleaned.columns)} columns, {len(df_cleaned)} records.")

### Spatial Feature Engineering

This step extracts borough-level road networks from OpenStreetMap and calculates betweenness and degree centrality to capture spatial structure in the transport network.

In [ ]:
# RoadCentrality
path = "../data/Borough_Boundaries.geojson"
boroughs = gpd.read_file(path)
boroughs = boroughs[["name", "gss_code", "geometry"]].rename(columns={"name": "borough"})

ox.settings.log_console = False
ox.settings.use_cache = True

results = []

for idx, row in tqdm(boroughs.iterrows(), total=len(boroughs), desc="Processing boroughs"):
    borough_name = row["borough"]
    gss_name = row["gss_code"]   
    geometry = row["geometry"]

    try:
        print(f"Processing: {borough_name}")
        
        G = ox.graph_from_polygon(geometry, network_type="drive", simplify=True)

        betweenness = nx.betweenness_centrality(G, weight="length", k=100, seed=42)
        degree = dict(G.degree())
        nx.set_node_attributes(G, betweenness, "betweenness")
        nx.set_node_attributes(G, degree, "degree")

        edge_data = []
        for u, v, key, data in G.edges(keys=True, data=True):
            edge_data.append({
                "u": u,
                "v": v,
                "key": key,
                "geometry": data.get("geometry", None),
                "betweenness": (G.nodes[u]["betweenness"] + G.nodes[v]["betweenness"]) / 2,
                "degree": (G.nodes[u]["degree"] + G.nodes[v]["degree"]) / 2
            })
        edges_df = gpd.GeoDataFrame(edge_data, geometry="geometry", crs="EPSG:4326")

        summary = {
            "borough": borough_name,
            "gss_code": gss_name,
            "mean_betweenness": edges_df["betweenness"].mean(),
            "max_betweenness": edges_df["betweenness"].max(),
            "mean_degree": edges_df["degree"].mean(),
            "max_degree": edges_df["degree"].max(),
            "edge_count": len(edges_df)
        }
        results.append(summary)

    except Exception as e:
        print(f"Failed for {borough_name}: {e}")
        continue

df_results = pd.DataFrame(results)
df_results.to_csv("../data/london_borough_road_centrality.csv", index=False)
print("All done! Results saved to 'london_borough_road_centrality.csv'")

In [ ]:
# show
print("Sample of calculated borough-level centrality metrics:")
display(df_results.head())

# Display descriptive statistical information
print("\nSummary statistics of centrality metrics across boroughs:")
display(df_results[['mean_betweenness', 'mean_degree']].describe())

### Data Merge & Summary

In [ ]:
# Set the path
accident_path = "../data/clean/1519_cleaned.csv"
centrality_path = "../data/london_borough_road_centrality.csv"
output_path = "../data/final/2015_2019_with_centrality.csv"

df_accident = pd.read_csv(accident_path)
df_centrality = pd.read_csv(centrality_path)

# Merge the centrality data (encoded by region)
df_merged = df_accident.merge(
    df_centrality,
    how="left",
    left_on="local_authority_ons_district",
    right_on="gss_code"
)

# Delete the rows lacking centrality (non-London area)
before_drop = len(df_merged)
df_merged = df_merged.dropna(subset=["mean_betweenness"])
after_drop = len(df_merged)
dropped = before_drop - after_drop

# Save the result
df_merged.to_csv(output_path, index=False)

print(f"The data has been combined with the centrality indicators and saved to：{output_path}")
print(f"Total: {after_drop} records, remove {dropped} records.")


### Exploratory Data Analysis (EDA)
#### Descriptive statistics (distribution maps, box plots, etc.)

In [ ]:
df = pd.read_csv("../data/final/2015_2019_with_centrality.csv")

print(df.shape)
print(df.dtypes)
print(df.isnull().sum()) 
df.describe()
df["accident_severity"].value_counts(normalize=True)

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Distribution of accident severity
plt.figure(figsize=(6,4))
sns.countplot(x="accident_severity", data=df)
plt.title("Accident Severity Distribution")
plt.show()

# Numerical type: Number of vehicles, number of casualties, speed limit
for col in ["number_of_vehicles", "number_of_casualties"]:
    plt.figure(figsize=(6, 4))
    sns.histplot(np.log1p(df[col]), kde=True)
    plt.title(f"Log-transformed distribution of {col}")
    plt.xlabel(f"log1p({col})")
    plt.ylabel("Count")
    plt.show()

plt.figure(figsize=(6, 4))
sns.histplot(df["speed_limit"], kde=True)
plt.title("Distribution of Speed Limit")
plt.xlabel("speed_limit")
plt.ylabel("Count")
plt.show()

In [ ]:
# Central variable distribution (single variable + null value check)
for col in ["mean_betweenness", "max_betweenness", "mean_degree", "max_degree"]:
    sns.histplot(df[col].dropna(), kde=True)
    plt.title(f"Distribution of {col}")
    plt.show()

#### Exploration of the Relationship between Features and Targets (including grouped bar charts and box plots)

In [ ]:
# The relationship between Variables and the severity of accidents (bivariate Analysis)
for col in ["mean_betweenness", "max_betweenness", "mean_degree", "max_degree"]:
    plt.figure(figsize=(6, 4))
    sns.boxplot(x="accident_severity", y=col, data=df)
    plt.title(f"{col} vs Accident Severity")
    plt.xlabel("Accident Severity (1 = Fatal, 2 = Serious, 3 = Slight)")
    plt.ylabel(col)
    plt.show()

# Divide mean_betweenness into four grades (quartiles)
df["betweenness_level"] = pd.qcut(df["mean_betweenness"], q=4, labels=["Low", "Medium-Low", "Medium-High", "High"])

# Check the proportion of accident severity in each group
severity_by_level = pd.crosstab(df["betweenness_level"], df["accident_severity"], normalize='index')

# Draw a grouped stacked bar chart
severity_by_level.plot(kind="bar", stacked=True, colormap="Set2")
plt.title("Accident Severity by Mean Betweenness Level")
plt.xlabel("Mean Betweenness Group")
plt.ylabel("Proportion of Accident Severity")
plt.legend(title="Severity", loc="upper right")
plt.show()

# Categorical variables can be analyzed in cross-tables:
pd.crosstab(df["day_of_week"], df["accident_severity"], normalize='index').plot(kind='bar', stacked=True)
plt.title("Accident Severity by Day of Week")

According to the boxplots and grouped bar charts, maximum betweenness centrality (max_betweenness) shows stronger differentiation across accident severity levels, especially with higher values in fatal accidents. In contrast, mean_betweenness exhibits weaker variation, indicating a more subtle influence. Degree-based indicators, particularly max_degree, show very limited discriminative power and may not be useful in predictive modeling.

## Methodology

[[ go back to the top ]](#Table-of-contents)

### Methodological Flow Chart of Data Integration, Feature Engineering, Model Training, and Interpretation


![image.png](../pic/flowchart.png)

### Modeling Preparation (Feature Engineering)

#### Feature Encoding

In [ ]:
# One-hot encoding + save
categorical_vars = [
    'day_of_week', 'road_type', 'light_conditions', 'weather_conditions',
    'road_surface_conditions', 'junction_control', 'junction_detail',
    'pedestrian_crossing_human_control', 'pedestrian_crossing_physical_facilities',
    'special_conditions_at_site', 'first_road_class',
    'second_road_class',
    'trunk_road_flag', 'urban_or_rural_area'
]

# Coding
df_encoded = pd.get_dummies(df.copy(), columns=categorical_vars, drop_first=True)

# Convert the Boolean column to an integer
for col in df_encoded.columns:
    if df_encoded[col].dtype == 'bool':
        df_encoded[col] = df_encoded[col].astype(int)

# Check the distribution of data types
print("Column types:\n", df_encoded.dtypes.value_counts())

# get hour
df_encoded["time_hour"] = pd.to_datetime(df_encoded["time"], format="%H:%M", errors="coerce").dt.hour

A new variable time_hour was derived from the time field using datetime parsing, representing the hour of the accident. Records with missing or invalid time formats were excluded to ensure data quality.

In [ ]:
# Ordinal encoding betweenness_level
betweenness_mapping = {
    'Low': 0,
    'Medium-Low': 1,
    'Medium-High': 2,
    'High': 3
}
df_encoded['betweenness_level_encoded'] = df_encoded['betweenness_level'].map(betweenness_mapping)
df_encoded.drop(columns=['betweenness_level'], inplace=True)

# Delete the fields that cannot be modeled
df_encoded.drop(columns=['time', 'borough', 'gss_code','local_authority_ons_district'], inplace=True)
# Delete the post hoc variable
df_encoded = df_encoded.drop(columns=['did_police_officer_attend_scene_of_accident', 'number_of_vehicles','number_of_casualties', 'carriageway_hazards'])
print(df_encoded.columns)

df_encoded.to_csv("../data/final/encode201519.csv", index=False)
print("Data saved to '../data/final/encode_all_years_with_centrality.csv'")

All categorical variables were either one-hot encoded or ordinal-encoded. The time variable was converted to time_hour, and betweenness_level was ordinally mapped to an integer scale. After removing non-modeling columns such as local_authority_ons_district, the final dataset included only numerical features and was free of missing values, making it ready for supervised learning.

#### Feature Selection & Drop

In [ ]:
import pandas as pd

df = pd.read_csv("../data/final/encode201519.csv")

# View the basic structure
print("DataFrame Info:")
print(df.info())

# Missing value check
print("\nMissing Values:")
missing = df.isnull().sum()
print(missing[missing > 0].sort_values(ascending=False))

# Data type statistics
print("\nData type distribution:")
print(df.dtypes.value_counts())

# Check the object type field
print("\nObject Type fields and the number of their unique values:")
obj_cols = df.select_dtypes(include='object')
print(obj_cols.nunique().sort_values(ascending=False))

The dataset used for modeling consists of 115,805 records with 97 numeric features after preprocessing. To simulate real-world forecasting, a temporal train-test split was adopted. Accidents from 2015–2018 were used for training, while 2019 data served as the hold-out test set. This temporal split ensures that the evaluation reflects the model's ability to generalize to future, unseen cases, rather than relying on random shuffling which may result in data leakage.

In [ ]:
# Construct features and labels
X = df.drop(columns=["accident_severity", "accident_year"])
y = df["accident_severity"]

# Divide the training set and the test set by year
X_train = X[df["accident_year"].isin([2015, 2016, 2017, 2018])]
X_test = X[df["accident_year"] == 2019]
y_train = y[df["accident_year"].isin([2015, 2016, 2017, 2018])]
y_test = y[df["accident_year"] == 2019]

In [ ]:
# Define the evaluation function
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

def evaluate_model(model, X_test, y_test, name="Model"):
    y_pred = model.predict(X_test)
    print(f"\n🔍 {name} Classification Report")
    print(classification_report(y_test, y_pred))

    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(6, 4))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(f"{name} - Confusion Matrix")
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.tight_layout()
    plt.show()

### Modeling and Evaluation

All three models were optimized using grid search with 3-fold cross-validation on the training set, based on macro-averaged F1-score as the evaluation metric. Macro-F1 is particularly appropriate for imbalanced multi-class classification problems, as it gives equal weight to each class regardless of sample size. This choice of scoring metric ensures that the models are not disproportionately tuned to the majority class performance, but instead maintain balanced treatment across all severity levels.

#### Logistic Regression

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Pipeline (Standardization + Logistic Regression)
logreg_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('logreg', LogisticRegression(max_iter=5000, random_state=42))
])

# Parameter grid
logreg_param_grid = {
    'logreg__C': [0.01, 0.1, 1, 10],
    'logreg__class_weight': ['balanced', None],
    'logreg__multi_class': ['multinomial'],
    'logreg__solver': ['lbfgs']
}

# Grid search
grid_search_logreg = GridSearchCV(
    logreg_pipeline,
    logreg_param_grid,
    scoring='f1_macro',
    cv=3,
    verbose=2,
    n_jobs=-1
)

# Train
grid_search_logreg.fit(X_train, y_train)
print("Logistic Regression Optimal parameters:", grid_search_logreg.best_params_)
print("Logistic Regression The best macro-F1 score:", grid_search_logreg.best_score_)

# Prediction + Visualization
y_pred_log = grid_search_logreg.best_estimator_.predict(X_test)
print("\nLogistic Regression Classification Report")
print(classification_report(y_test, y_pred_log))

# Confusion matrix
cm_log = confusion_matrix(y_test, y_pred_log)
plt.figure(figsize=(6, 4))
sns.heatmap(cm_log, annot=True, fmt='d', cmap='Blues')
plt.title("Logistic Regression - Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.tight_layout()
plt.show()

#### Random Forest

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

# Create pipeline
rf_pipeline = Pipeline([
    ('scaler', StandardScaler()), 
    ('rf', RandomForestClassifier(random_state=42, n_jobs=-1))
])

# Parameter grid
rf_param_grid = {
    'rf__n_estimators': [100, 300],
    'rf__max_depth': [10, 20, None],
    'rf__min_samples_split': [2, 5],
    'rf__class_weight': ['balanced', None]
}

# Grid search
grid_search_rf = GridSearchCV(
    rf_pipeline,
    rf_param_grid,
    scoring='f1_macro',
    cv=3,
    verbose=2,
    n_jobs=-1
)

# Train
grid_search_rf.fit(X_train, y_train)

# Output result
print("RF Optimal parameters:", grid_search_rf.best_params_)
print("RF The best macro-F1 score:", grid_search_rf.best_score_)

# Evaluation
from sklearn.metrics import classification_report

y_pred_rf = grid_search_rf.best_estimator_.predict(X_test)
print(classification_report(y_test, y_pred_rf))

cm_rf = confusion_matrix(y_test, y_pred_rf)
plt.figure(figsize=(6, 4))
sns.heatmap(cm_rf, annot=True, fmt='d', cmap='Reds')
plt.title("Random Forest - Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.tight_layout()
plt.show()

#### XGBoost

In [ ]:
# XGBoost
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Create XGBoost pipeline
xgb_pipeline = Pipeline([
    ('scaler', StandardScaler()), 
    ('xgb', XGBClassifier(objective='multi:softprob', eval_metric='mlogloss', random_state=42, use_label_encoder=False))
])

# Parameter grid
xgb_param_grid = {
    'xgb__n_estimators': [100, 200],
    'xgb__max_depth': [6, 10],
    'xgb__learning_rate': [0.05, 0.1],
    'xgb__subsample': [0.8, 1.0]
}

# Grid search
grid_search_xgb = GridSearchCV(
    xgb_pipeline,
    xgb_param_grid,
    scoring='f1_macro',
    cv=3,
    verbose=2,
    n_jobs=-1
)

y_train = y_train - 1
y_test = y_test - 1

# Train
grid_search_xgb.fit(X_train, y_train)

# outcome
print("✅ XGB Optimal parameters:", grid_search_xgb.best_params_)
print("✅ XGB The best macro-F1 score:", grid_search_xgb.best_score_)

# Prediction + Visualization
y_pred_xgb = grid_search_xgb.best_estimator_.predict(X_test)
print(classification_report(y_test, y_pred_xgb))

cm_xgb = confusion_matrix(y_test, y_pred_xgb)
plt.figure(figsize=(6, 4))
sns.heatmap(cm_xgb, annot=True, fmt='d', cmap='Blues')
plt.title("XGBoost - Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.tight_layout()
plt.show()


### Model Comparison and Final Selection

Three supervised learning models were implemented to classify the severity of road traffic accidents: Logistic Regression, Random Forest, and XGBoost. Each model was evaluated based on its ability to capture class imbalance and distinguish between fatal, serious, and slight outcomes.

Logistic Regression achieved high overall accuracy (0.88) but failed to correctly identify any fatal or serious cases, leading to a low macro-F1 score and no practical utility in real-world accident prevention.

XGBoost offered improved performance over Logistic Regression in terms of macro-F1 and recall for the serious class, but remained heavily biased toward the majority class.

Random Forest delivered the best overall balance between interpretability and performance, with a macro-F1 score of 0.35 and noticeably higher recall on minority classes. It also demonstrated stable results during cross-validation and allowed post-hoc interpretation using SHAP.


#### Table: Comparison of Model Performance
| Model               | Accuracy | Macro F1 | Precision (avg) | Recall (avg) | F1-score (avg) | Notable Issues                           |
|--------------------|----------|----------|------------------|---------------|----------------|------------------------------------------|
| Logistic Regression | 0.88     | 0.31     | 0.81             | 0.65          | 0.72           | Completely failed to detect fatal/serious |
| Random Forest       | 0.81     | 0.35     | 0.75             | 0.68          | 0.72           | Most balanced, interpretable              |
| XGBoost             | 0.85     | 0.32     | 0.79             | 0.66          | 0.72           | Still biased toward majority class        |



Given these results, Random Forest was selected as the final model for its superior trade-off between classification performance and interpretability. Its output was further analysed using SHAP values, revealing that temporal and spatial network features were among the most influential predictors of accident severity.

### Model interpretation
#### Grouped Feature Importances (based on RF)

In [ ]:
import re
from collections import defaultdict
import pandas as pd
import matplotlib.pyplot as plt

# Extract the RF part of the best model from the trained GridSearch
rf_model = grid_search_rf.best_estimator_.named_steps['rf']

# Use the column names of the training set as feature names
feature_names = X_train.columns.tolist()

# Obtain the importance of features
importances = rf_model.feature_importances_

# Group Aggregation importance
grouped_importance = defaultdict(float)

for feat, imp in zip(feature_names, importances):
    match = re.match(r"(.+?)_(\d+)$", feat)
    if match:
        base_feat = match.group(1)
    else:
        base_feat = feat
    grouped_importance[base_feat] += imp

# trans to DataFrame
grouped_df = pd.DataFrame({
    'Feature Group': list(grouped_importance.keys()),
    'Total Importance': list(grouped_importance.values())
}).sort_values(by='Total Importance', ascending=False)

# visualize top 10
plt.figure(figsize=(10, 6))
plt.barh(grouped_df['Feature Group'][:10][::-1], grouped_df['Total Importance'][:10][::-1])
plt.xlabel("Aggregated Importance")
plt.title("Top 10 Grouped Feature Importances (Random Forest)")
plt.tight_layout()
plt.show()

# export
print(grouped_df.head(10))


#### SHAP

In [ ]:
import shap
best_pipeline_rf = grid_search_rf.best_estimator_
rf_model = best_pipeline_rf.named_steps['rf']
X_train_raw = X_train.copy()
explainer = shap.Explainer(rf_model, X_train_raw)

# This step will cost about 45 min
shap_values = explainer(X_train_raw)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

mean_abs_shap = np.abs(shap_values.values).mean(axis=(0, 2))  # shape: (85,)

feature_names = X_train_raw.columns
assert len(mean_abs_shap) == len(feature_names), "Mismatch between SHAP values and feature names"

shap_df = pd.DataFrame({
    'feature': feature_names,
    'mean_abs_shap': mean_abs_shap
})

def get_base_feature(f):
    parts = f.split('_')
    if parts[-1].isdigit() and len(parts) > 2:
        return '_'.join(parts[:-1])
    elif parts[-1].isdigit():
        return parts[0]
    return f

shap_df['base_feature'] = shap_df['feature'].apply(get_base_feature)

grouped_shap = shap_df.groupby('base_feature')['mean_abs_shap'].sum().reset_index()
grouped_shap = grouped_shap.sort_values(by='mean_abs_shap', ascending=False)

plt.figure(figsize=(10, 6))
plt.barh(grouped_shap['base_feature'], grouped_shap['mean_abs_shap'])
plt.xlabel('Mean(|SHAP value|)')
plt.title('Grouped SHAP Importance (by Feature Group)')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

In [ ]:
shap.summary_plot(shap_values, X_train_raw)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

mean_abs_shap = np.abs(shap_values.values).mean(axis=(0, 2))  # shape: (85,)

feature_names = X_train_raw.columns
assert len(mean_abs_shap) == len(feature_names), "Mismatch between SHAP values and feature names"

shap_df = pd.DataFrame({
    'feature': feature_names,
    'mean_abs_shap': mean_abs_shap
})

def get_base_feature(f):
    parts = f.split('_')
    if parts[-1].isdigit() and len(parts) > 2:
        return '_'.join(parts[:-1])
    elif parts[-1].isdigit():
        return parts[0]
    return f

shap_df['base_feature'] = shap_df['feature'].apply(get_base_feature)

grouped_shap = shap_df.groupby('base_feature')['mean_abs_shap'].sum().reset_index()
grouped_shap = grouped_shap.sort_values(by='mean_abs_shap', ascending=False)

plt.figure(figsize=(10, 6))
plt.barh(grouped_shap['base_feature'], grouped_shap['mean_abs_shap'])
plt.xlabel('Mean(|SHAP value|)')
plt.title('Grouped SHAP Importance (by Feature Group)')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

In [ ]:
feature_names = X_train_raw.columns

class_names = ['Fatal', 'Serious', 'Slight']
shap_grouped_by_class = {cls: defaultdict(float) for cls in class_names}

for class_idx, class_label in enumerate(class_names):
    shap_vals = shap_values.values[:, :, class_idx]
    shap_mean_abs = np.abs(shap_vals).mean(axis=0)

    for feat_name, shap_val in zip(feature_names, shap_mean_abs):
        match = re.match(r"(.+?)_(\d+)$", feat_name)
        base_feat = match.group(1) if match else feat_name
        shap_grouped_by_class[class_label][base_feat] += shap_val

all_features = sorted(set().union(*[d.keys() for d in shap_grouped_by_class.values()]))
df_plot = pd.DataFrame({
    'Feature Group': all_features,
    'Fatal': [shap_grouped_by_class['Fatal'].get(f, 0) for f in all_features],
    'Serious': [shap_grouped_by_class['Serious'].get(f, 0) for f in all_features],
    'Slight': [shap_grouped_by_class['Slight'].get(f, 0) for f in all_features]
})

df_plot['Total'] = df_plot['Fatal'] + df_plot['Serious'] + df_plot['Slight']
df_top10 = df_plot.sort_values(by='Total', ascending=False).head(10)

x = np.arange(len(df_top10['Feature Group']))
width = 0.25

plt.figure(figsize=(12, 6))
plt.bar(x - width, df_top10['Fatal'], width, label='Fatal', color='royalblue')
plt.bar(x, df_top10['Serious'], width, label='Serious', color='deeppink')
plt.bar(x + width, df_top10['Slight'], width, label='Slight', color='olive')

plt.xticks(x, df_top10['Feature Group'], rotation=45, ha='right')
plt.ylabel('Mean(|SHAP value|)')
plt.title('Top 10 Grouped SHAP Importances by Class')
plt.legend()
plt.tight_layout()
plt.show()


SHAP (SHapley Additive exPlanations) was employed to interpret model predictions and evaluate feature importance at both global and class-specific levels. This method is grounded in cooperative game theory and attributes model output to each input feature fairly and consistently. The global SHAP summary plots reveal the average magnitude of feature contributions across all predictions, whereas the class-wise SHAP decomposition further isolates the feature influence on fatal, serious, and slight injuries separately. This multi-layered interpretation enhances model transparency and facilitates policy-relevant insights.

## Results and discussion

[[ go back to the top ]](#Table-of-contents)

Three supervised learning models were trained to classify accident severity: Logistic Regression, Random Forest, and XGBoost, using a dataset of **115,805 records** from 2015–2019 with **97 numerical features**. The target variable had a highly imbalanced distribution: **“slight” = 87.6%**, **“serious” = 11.9%**, and **“fatal” = 0.5%**, making **macro-F1 and per-class recall** more appropriate than accuracy for evaluation.

- **Logistic Regression** achieved the highest overall accuracy (**0.88**), but failed to detect any “fatal” or “serious” cases (macro-F1 = **0.31**).  
- **XGBoost** offered slightly better recall for the “serious” class and achieved a macro-F1 of **0.32**, but remained heavily biased toward the majority class.  
- **Random Forest** delivered the most balanced performance, with an accuracy of **0.81** and a macro-F1 score of **0.35**, successfully identifying a subset of minority cases (recall: fatal = 0.02, serious = 0.08).

These results illustrate the risk of relying on accuracy in imbalanced classification. For example, **Kumar and Teja Santosh (2022)** reported 96.18% accuracy using XGBoost, but did not address class imbalance or report recall, making such models less reliable in safety-critical applications. This study emphasizes metrics that reflect model fairness across all classes.

Random Forest’s feature importance revealed that `time_hour`, `day_of_week`, and `max_betweenness` were consistently impactful. `Max_betweenness`, a spatial indicator derived from borough-level road network topology, appeared among the top 5 predictors, validating the integration of spatial structure into severity modeling.

To further interpret the model, **SHAP (SHapley Additive Explanations)** was applied. Global SHAP analysis confirmed the high contribution of temporal and spatial features. Class-specific SHAP bar plots showed that:
- For **fatal** accidents, the most influential features were `max_betweenness`, `speed_limit`, and `light_conditions_5` (dark, no street lighting).
- For **serious** accidents, `junction_detail`, `pedestrian_crossing_physical_facilities`, and `first_road_class` played larger roles.
- For **slight** accidents, `time_hour`, `weather_conditions`, and general road context were dominant.

These findings demonstrate that combining **spatial network metrics** with **contextual accident features** enhances both predictive accuracy and model interpretability for road safety applications.

## Conclusion

[[ go back to the top ]](#Table-of-contents)

This study investigated whether supervised machine learning models can accurately predict the severity of road traffic accidents in London, using spatial, temporal, and environmental features. A borough-level dataset from 2015 to 2019 was constructed, integrating UK accident records with spatial centrality indicators derived from OpenStreetMap.

Among the models tested, Random Forest demonstrated the best overall balance between performance and interpretability, achieving a macro-F1 score of 0.35. SHAP analysis further revealed that features such as `max_betweenness`, `time_hour`, and `road type` significantly contributed to severity predictions. These results suggest that incorporating spatial network metrics meaningfully enhances the capacity of data-driven safety models to identify severe accident risks.

However, this project has several limitations. The severe class imbalance in the dataset limited the model’s ability to generalize predictions for rare fatal cases. The spatial resolution was restricted to the borough level, which may obscure finer-scale local effects. In addition, the study only used tabular features; incorporating trajectory-level or vehicle-specific data could improve model fidelity.

Future work may explore finer spatial units, additional data modalities (e.g., street view imagery or traffic flow), and deep learning methods for enhanced prediction and interpretability.

## References

[[ go back to the top ]](#Table-of-contents)

Abdel-Aty, M. and Haleem, K. (2011) ‘Analyzing angle crashes at unsignalized intersections using machine learning techniques’, Accident Analysis & Prevention, 43(1), pp. 461–470. Available at: https://doi.org/10.1016/j.aap.2010.10.002.

Ahmed, S. et al. (2023) ‘A study on road accident prediction and contributing factors using explainable machine learning models: analysis and performance’, Transportation Research Interdisciplinary Perspectives, 19, p. 100814. Available at: https://doi.org/10.1016/j.trip.2023.100814.

Kumar, A.P. and Teja Santosh, D. (2022) ‘Road Accident Severity Prediction Using Machine Learning Algorithms’, International Journal of Computer Engineering in Research Trends, 9(9), pp. 175–183. Available at: https://doi.org/10.22362/ijcert/2022/v9/i9/v9i902.

Department for Transport (DfT), 2013. New penalties for careless driving come into force. [online] Available at: https://www.gov.uk/government/news/new-penalties-for-careless-driving-come-into-force [Accessed 20 Apr. 2025].

Department for Transport (DfT), 2021. Impacts of 2020 Low Traffic Neighbourhoods in London on Road Traffic Injuries. [pdf] Available at: https://assets.publishing.service.gov.uk/media/65f400adfa18510011011787/low-traffic-neighbourhoods-research-report.pdf [Accessed 20 Apr. 2025].

Transport for London (TfL), 2024. The impacts of Low Traffic Neighbourhoods in London. [pdf] Available at: https://tfl.gov.uk/cdn/static/cms/documents/tfl-impacts-of-low-traffic-neighbourhoods-feb-2024-acc.pdf [Accessed 20 Apr. 2025].